# FloPy 

### MODFLOW-2005 based model checker demonstration

In [1]:
import os
import sys
from tempfile import TemporaryDirectory

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join("..", ".."))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print("flopy version: {}".format(flopy.__version__))

3.7.15 (default, Oct 18 2022, 09:36:49) 
[GCC 11.2.0]
flopy version: 3.3.6


#### Set the working directory

In [2]:
path = os.path.join("..", "data", "mf2005_test")

#### Load example dataset and change the model work space

In [3]:
m = flopy.modflow.Modflow.load("test1ss.nam", model_ws=path)
temp_dir = TemporaryDirectory()
workspace = temp_dir.name
m.change_model_ws(workspace)

By default, the checker performs a model-level check when a set of model files are loaded, unless load is called with `check=False`. The load check only produces screen output if load is called with `verbose=True`. Checks are also performed at the package level when an individual package is loaded

#### The `check()` method
Each model and each package has a `check()` method. The check method has three arguments:

In [4]:
help(m.check)

Help on method check in module flopy.mbase:

check(f=None, verbose=True, level=1) method of flopy.modflow.mf.Modflow instance
    Check model data for common errors.
    
    Parameters
    ----------
    f : str or file handle
        String defining file name or file handle for summary file
        of check method output. If a string is passed a file handle
        is created. If f is None, check method does not write
        results to a summary file. (default is None)
    verbose : bool
        Boolean flag used to determine if check method results are
        written to the screen
    level : int
        Check method analysis level. If level=0, summary checks are
        performed. If level=1, full checks are performed.
    
    Returns
    -------
    None
    
    Examples
    --------
    
    >>> import flopy
    >>> m = flopy.modflow.Modflow.load('model.nam')
    >>> m.check()



#### The check class

By default, check is called at the model level without a summary output file, but with `verbose=True` and `level=1`. The check methods return an instance of the **check** class, which is housed with the flopy utilities.

In [5]:
chk = m.check()


test1ss MODEL DATA VALIDATION SUMMARY:
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: thin cells (less than checker threshold of 1.0)
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    LPF package: zero or negative horizontal hydraulic conductivity values
    LPF package: negative horizontal anisotropy values
    LPF package: vertical hydraulic conductivity values below checker threshold of 1e-11
    LPF package: vertical hydraulic conductivity values above checker threshold of 100000.0
    LPF pac

#### Summary array
Most of the attributes and methods in **check** are intended to be used by the ``check()`` methods. The central attribute of **check** is the summary array:

In [6]:
chk.summary_array

rec.array([('Warning', 'OC', 0, 0, 0, 0.0000000e+00, '\r    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS'),
           ('Warning', 'RCH', 0, 0, 0, 1.1785914e-09, '\r    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods'),
           ('Warning', 'RCH', 0, 0, 0, 1.0000000e+00, '\r    RCH package: Variable NRCHOP set to value other than 3')],
          dtype=[('type', 'O'), ('package', 'O'), ('k', '<i8'), ('i', '<i8'), ('j', '<i8'), ('value', '<f8'), ('desc', 'O')])

This is a numpy record array that summarizes errors and warnings found by the checker. The package, layer-row-column location of the error, the offending value, and a description of the error are provided. In the checker, errors and warnings are loosely defined as follows:  
#### Errors:

either input that would cause MODFLOW to crash, or inputs that almost certainly mis-represent the intended conceptual model.

#### Warnings:

inputs that are potentially problematic, but may be intentional.

each package-level check produces a **check** instance with a summary array. The model level checks combine the summary arrays from the packages into a master summary array. At the model and the package levels, the summary array is used to generate the screen output shown above. At either level, the summary array can be written to a csv file by supply a filename to the `f` argument. Specifying `level=2` prints the summary array to the screen.

In [7]:
m.check(level=2)

Errors and/or Warnings encountered.


Errors and/or Warnings encountered.

test1ss MODEL DATA VALIDATION SUMMARY:
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RCH package: Variable NRCHOP set to value other than 3

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: thin cells (less than checker threshold of 1.0)
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    LPF package: zero or negative horizontal hydraulic conductivity values
    LPF package: negative horizontal anisotropy values
    LPF package: vertical hydraulic conductivity values below checker threshold of 1e-11
    LPF package: vertical hydraulic conductivity values above check

#### example of package level check and summary file

In [8]:
m.rch.check()


RCH PACKAGE DATA VALIDATION:
  2 Warnings:
    Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    Variable NRCHOP set to value other than 3



#### example of summary output file

In [9]:
m.check(f=os.path.join(workspace, "checksummary.csv"))


test1ss MODEL DATA VALIDATION SUMMARY:
  3 Warnings:
    OC package: action(s) defined in OC stress_period_data ignored as they are not part the stress periods defined by DIS
    RCH package: Mean R/T ratio < checker warning threshold of 2e-08 for 1 stress periods
    RCH package: Variable NRCHOP set to value other than 3
  see /tmp/tmp46iib3r0/checksummary.csv for details.

  Checks that passed:
    Unit number conflicts
    Compatible solver package
    DIS package: zero or negative thickness
    DIS package: thin cells (less than checker threshold of 1.0)
    DIS package: nan values in top array
    DIS package: nan values in bottom array
    BAS6 package: isolated cells in ibound array
    BAS6 package: Not a number
    LPF package: zero or negative horizontal hydraulic conductivity values
    LPF package: negative horizontal anisotropy values
    LPF package: vertical hydraulic conductivity values below checker threshold of 1e-11
    LPF package: vertical hydraulic conductivity v

In [10]:
summary_pth = os.path.join(workspace, "checksummary.csv")

try:
    import pandas as pd

    df = pd.read_csv(summary_pth)
except:
    df = open(summary_pth).readlines()
df

,type,package,k,i,j,value,desc
0,Warning,OC,0,0,0,0.0,action(s) defined in OC stress_period_data ign...
1,Warning,RCH,0,0,0,0.0,Mean R/T ratio < checker warning threshold of ...
2,Warning,RCH,0,0,0,1.0,Variable NRCHOP set to value other than 3


#### checking on `write_input()`
checking is also performed by default when `write_input()` is called at the package or model level. Checking on write is performed with the same `verbose` setting as specified for the model. However, if errors or warnings are encountered and `level=1` (default) or higher, a screen message notifies the user of the errors.

By default, the checks performed on `load()` and `write_input()` save results to a summary file, which is named after the packge or the model.

In [11]:
m.write_input()